## Préparation des données pour les modèles DIST_REL_CC, DIST_REL_CH et DIST_REL_OH

### Intention

À la lumière des informations extraites de l'analyse du modèle DIST_REL_C_01 (voir notebook 11.1), nous allons tenter de généraliser la méthode à plusieurs types de liaisons :

* carbone-hydrogène (modèle DIST_REL_CH)
* oxygène-hydrogène (modèle DIST_REL_OH)
* carbone-carbone (modèle DIST_REL_CC)

Le modèle DIST_REL_CC est donc identique au modèle DIST_REL_C_01, à la différence qu'on va l'entraîner sur plus d'exemples.


### Méthodologie

Nous allons de ce notebook générer les données d'entraînement pour chacun des modèles. Ces données auront les caractéristiques suivantes :

* Extraites de molécules ne contenant pas d'atome plus lourd que le fluor
* Extraites de molécules contenant entre 2 et 60 atomes
* Les données sur les molécules sont des données convergées

Chaque exemple du jeu de données correspondra aux informations concerant la longueur de la liaison entre deux atomes C-C, C-H ou O-H (en fonction du modèle associé au jeu de données).

* Données d'entrée :
    * Pour chaque exemple :
        * Pour chaque atome n'intervenant pas dans la liaison :
            * Le numéro atomique (encodé en one-hot-encoding)
            * La masse atomique
            * La classe positionnelle de l'atome par rapport à la liaison (voir notebook 9.1)
            * La distance à chacun des deux atomes de la liaison
            
            

* Données de sortie (cibles) :
    * Pour chaque exemple :
        * La distance entre les deux atomes de la liaison considérée
        

Une fois les jeux de données créés, nous entraîner un modèle par jeu de données, puis nous analyserons les prédictions de chacun des modèles sur des données de test que les modèles n'auront jamais vues (et que nous crééons également dans ce notebook).




#### Chemin des fichiers

In [1]:
# Données moléculaires d'origine
train_riken_reduced_loc = "../data/train_set_riken_v2_reduced.h5"
test_riken_reduced_loc = "../data/test_set_riken_v2_reduced.h5"

# Jeux préparés pour le modèle DIST_REL_CC
train_CC_prepared_input_loc = "../data/DIST_REL_CC/train_set_prepared_input.h5"
train_CC_labels_loc = "../data/DIST_REL_CC/train_set_labels.h5"
test_CC_prepared_input_loc = "../data/DIST_REL_CC/test_set_prepared_input.h5"
test_CC_labels_loc = "../data/DIST_REL_CC/test_set_labels.h5"

# Jeux préparés pour le modèle DIST_REL_CH
train_CH_prepared_input_loc = "../data/DIST_REL_CH/train_set_prepared_input.h5"
train_CH_labels_loc = "../data/DIST_REL_CH/train_set_labels.h5"
test_CH_prepared_input_loc = "../data/DIST_REL_CH/test_set_prepared_input.h5"
test_CH_labels_loc = "../data/DIST_REL_CH/test_set_labels.h5"

# Jeux préparés pour le modèle DIST_REL_OH
train_OH_prepared_input_loc = "../data/DIST_REL_OH/train_set_prepared_input.h5"
train_OH_labels_loc = "../data/DIST_REL_OH/train_set_labels.h5"
test_OH_prepared_input_loc = "../data/DIST_REL_OH/test_set_prepared_input.h5"
test_OH_labels_loc = "../data/DIST_REL_OH/test_set_labels.h5"

# Jeux de données pour tests en local
minimal_reduced_loc = "../data/minimal_set_riken_v2_reduced.h5"
minimal_prepared_input_loc = "../data/minimal_prepared_input.h5"
minimal_labels_loc = "../data/minimal_labels.h5"

#### Constantes

On définit ici les constantes concernant les numéros atomiques des atomes que l'on considère, et concernant les distances en deçà desquelles on considère qu'un couple d'atome partage une liaison. Ces distances sont calculées de la façon suivante : moyenne de la liaison + quatre fois l'écart-type.

In [4]:
H_ANUM = 1.
C_ANUM = 6.
O_ANUM = 8.

DIST_MAX_CC = 1.6
DIST_MAX_CH = 1.35
DIST_MAX_OH = 1.25

#### Calcul de la distance entre deux atomes

In [3]:
import numpy as np

def calcul_distance(pt1, pt2):
    """ Renvoie la distance entre deux points représentés par leurs coordonnées (x, y, z) dans deux tableaux
    de forme (1, 3)"""
    return np.sqrt(np.sum(np.square(np.diff(np.array([pt1, pt2]), axis=0))))

#### Fonction de test si deux atomes partagent une liaison (en fonction de leurs positions et numéros atomiques)

In [9]:
def existe_liaison(pos1, anum1, pos2, anum2):
    
    return ((anum1==C_ANUM and anum2==C_ANUM and calcul_distance(pos1, pos2) <= DIST_MAX_CC) or
            (anum1==C_ANUM and anum2==H_ANUM and calcul_distance(pos1, pos2) <= DIST_MAX_CH) or
            (anum1==O_ANUM and anum2==H_ANUM and calcul_distance(pos1, pos2) <= DIST_MAX_OH))

#### Fonction renvoyant les distances d'un atome à deux autres atomes

In [10]:
def calcul_distances(pt1, pt_ref1, pt_ref2):
    return np.array([calcul_distance(pt1, pt_ref1), calcul_distance(pt1, pt_ref2)])

#### Fonction renvoyant la classe positionnelle d'un atome en fonction de sa position et des positions des deux atomes de référence. Renvoie le résultat sous forme de one-hot-encoding

In [11]:
def get_classe_pos(pt, pos_a1, pos_a2):
    
    # Initialisation du tableau de sortie
    classes_pos = np.zeros(shape=(3,))
    
    # Calcul du vecteur A1_A2
    vect = np.diff([pos_a1, pos_a2], axis=0)
    
    # Déclaration de la matrice contenant les coordonnées des trois points g, c et d
    gcd_pos = np.empty(shape=(3, 3))
    
    # Calcul du point c
    gcd_pos[1] = np.divide([np.sum([pos_a1, pos_a2], axis=0)], 2)

    # Calcul des points g et d
    gcd_pos[0] = gcd_pos[1] - vect
    gcd_pos[2] = gcd_pos[1] + vect
        
    # Calcul de la matrice des positions répétées du point
    pos = np.tile(pt, 3).reshape(3, 3)
    
    # Calcul des distances du point avec les points g, c et d
    dists = np.sqrt(np.sum(np.square(np.diff([gcd_pos, pos], axis=0)[0]), axis=1))
    
    # On renvoie le résultat encodé avec la méthode du one hot encoding
    classes_pos[np.argmin(dists)] = 1
    return classes_pos
    

#### Fonction renvoyant le numéro de l'atome encodé en one-hot encoding (pour les atomes des deux premières lignes du tableau périodique)

In [12]:
def get_anum_one_hot(z):
    
    if not 0 < z < 10:
        raise RuntimeError("Atomic number must be between 1 and 9")
    
    one_hot = np.zeros(shape=(9,))
    one_hot[int(z)-1] = 1
    return one_hot


#### Fonction renvoyant les données préparées pour le RN pour une molécule

In [14]:
def donnees_prep_RN_mol(coords_mol, anums_mol, amasses_mol, anum_1, anum_2):
    """ Fonction préparant les données à fournir aux modèles pour une molécule. On prend en entrée les
        données décrivant la molécules (coordonnées, numéros atomiques et masses atomiques) ainsi que
        les deux numéros atomiques des atomes dont on veut prédire la longueur de la liaison.
        On itère ensuite sur tous les couples de ces deux numéros atomiques, puis pour ceux qui partagent
        une liaison, pour chaque atome de la molécule privée des deux atomes du couple on créé une entrée
        du modèle composée du numéro atomique de l'atome (one-hot), de la classe positionnelle de l'atome
        (one-hot), de la masse atomique de l'atome et des distances de l'atome aux deux atomes du couple
        considéré.
        Pour chaque couple, la cible du modèle est la distance entre les deux atomes du couple """
    
    taille_mol = len(coords_mol)
    
    inputs_RN = []
    targets_RN = []
    
    # On itère sur tous les atomes de la molécule
    for i in range(taille_mol):
        
        # On sélectionne le premier atome du couple
        if anums_mol[i] == anum_1:
            
            # On parcourt tous les atomes suivants
            for j in range(i+1, taille_mol):
                
                # On sélectionne le second atome du couple s'il existe une liaison avec le premier
                if anums_mol[j] == anum_2 and existe_liaison_C(coords_mol[i], anum_1, coords_mol[j], anum_2):
                    
                    # On calcule la distance entre les deux atomes du couple
                    dist_couple = calcul_distance(coords_mol[i], coords_mol[j])
                    
                    # On initialise l'entrée du RN pour le couple courant
                    input_rn = np.empty(shape=(taille_mol-2, 15))
                    
                    input_rn_idx = 0
                    
                    # On itère sur tous les atomes de la molécule (en dehors des deux atomes du couple)
                    for k in range(taille_mol):
                        if k != i and k != j:
                            
                            # On enregistre le numéro et la masse atomique de l'atome
                            input_rn[input_rn_idx][:9] = get_anum_one_hot(anums_mol[k])
                            input_rn[input_rn_idx][9] = amasses_mol[k]
                            
                            # On enregistre les distances de l'atome aux deux atomes du couple
                            input_rn[input_rn_idx][10:12] = calcul_distances(coords_mol[k], coords_mol[i],
                                                                         coords_mol[j])
                            
                            # On enregistre la classe positionnelle de l'atome par rapport aux deux atomes
                            # de carbone
                            input_rn[input_rn_idx][12:15] = get_classe_pos(coords_mol[k], coords_mol[i], 
                                                                        coords_mol[j])    

                            input_rn_idx += 1
                            
                    # On aplatit l'entrée du RN
                    input_rn = np.array(input_rn)
                    input_rn = input_rn.reshape(-1,)
                            
                    # On enregistre l'entrée et la cible du RN pour le couple courant
                    inputs_RN.append(input_rn)
                    
                    # On enregistre la distance cible pour le couple d'atomes courant (en mÅ)
                    targets_RN.append(dist_couple*1000)   
                
    return np.array(inputs_RN), np.array(targets_RN)
            
        


#### Définition de la fonction de création du jeu de données

In [ ]:
import h5py
import time

def generer_donnees(original_dataset_loc, prepared_input_loc, labels_loc, anum1, anum2, nb_mol, batch_size):
    """ Génère les entrées préparées et les labels pour le jeu de données, pour les couples d'atomes
        possédant les numéros atomiques donnés, à partir des nb\_mol premières molécules du jeu.
        Enregistre en plus un fichier h5 contenant toutes les distances """
    
    start_time = time.time()    
    
    # On charge les données du jeu original en mémoire
    original_dataset_h5 = h5py.File(original_dataset_loc, "r")
    input_coords = np.array(original_dataset_h5["riken_coords"][:nb_mol])
    input_masses = np.array(original_dataset_h5["amasses"][:nb_mol])
    input_nums = np.array(original_dataset_h5["anums"][:nb_mol])
    
    # On créé les jeux de données d'entrée du RN et de labels
    input_rn_dataset_h5 = h5py.File(prepared_input_loc, 'w')
    labels_dataset_h5 = h5py.File(labels_loc, 'w')    
    
    try:
        varlen_floatarray = h5py.special_dtype(vlen=np.dtype("float32"))
        
        input_dataset = input_rn_dataset_h5.create_dataset("inputs", maxshape=(None,), shape=(0,),
                                           dtype=varlen_floatarray, compression="gzip", 
                                           chunks=True)

        target_dataset = labels_dataset_h5.create_dataset("targets", maxshape=(None,), shape=(0,),
                                           dtype=np.float32, compression="gzip", 
                                           chunks=True)


        datasets_curr_idx = 0
        
        # On créé les tableaux pour le premier batch de traitement
        inputs_batch = []
        outputs_batch = []

        # On parcourt toutes les molécules du jeu original
        for ori_idx in range(nb_mol):
            
            # On ne s'intéresse pas aux molécules contenenant des atomes plus lourds que le fluor (contrainte
            # imposée par l'utilisation d'un one-hot encoding pour les numéros atomiques)
            if max(input_nums[ori_idx])<10:
                
                # On calcule les entrées et sortie du RN pour la molécule courante (il peut y avoir plusieurs
                # entrées et plusieurs sorties s'il existe plusieurs couples de liaisons considérées
                # dans la molécule)
                curr_inputs_np, curr_outputs_np = donnees_prep_RN_mol(input_coords[ori_idx].reshape(-1, 3), 
                                                                    input_nums[ori_idx],
                                                                    input_masses[ori_idx])
                
                # On ajoute les entrées et sorties aux tableaux du batch courant
                inputs_batch.extend(curr_inputs_np)
                outputs_batch.extend(curr_outputs_np)
                   
            # On écrit les tableaux h5 du batch courant
            if ori_idx % batch_size == 0 or ori_idx == len(input_nums)-1:
                
                print("Molécule courante : "+str(ori_idx))
    
                # On redimensionne les datasets
                input_dataset.resize((input_dataset.shape[0]+len(inputs_batch),))
                target_dataset.resize((target_dataset.shape[0]+len(inputs_batch),))
                
                # On écrit les données dans les datasets
                input_dataset[datasets_curr_idx:datasets_curr_idx+len(inputs_batch)] = np.array(inputs_batch)
                target_dataset[datasets_curr_idx:datasets_curr_idx+len(inputs_batch)] = np.array(outputs_batch)
                
                datasets_curr_idx += len(inputs_batch)
                
                # On réinitialise les tableaux
                inputs_batch = []
                outputs_batch = []
                
        input_rn_dataset_h5.flush()
        labels_dataset_h5.flush()
        
        print(str(len(input_dataset))+" exemples créés")
        print("--- %s seconds ---" % (time.time() - start_time))
                
    
    finally:
        input_rn_dataset_h5.close()
        labels_dataset_h5.close()
        original_dataset_h5.close()

    
    